In [2]:
import pandas as pd
import numpy as np
import json 

In [3]:
def describe_df(df):
    '''
    Вывод простой статистки DataFrame
    '''
    desc = df.describe(include='all',percentiles=[0.5]).drop(index=['mean','std','50%']).T 
    desc['type'] = [type(x) for x in df.iloc[0]]
    desc['NaN_prop'] = round(df.isna().sum()/len(df),3)
    desc.unique = df.nunique()
    desc.top = df.mode(axis=0).iloc[0]
    desc.top = [str(x)[:30] for x in df.mode(axis=0).iloc[0]]

    desc.freq = [df[col].value_counts().iloc[0] for col in  df.columns]
    
    return desc

In [4]:
RANDOM_SEED = 42
path = './Project_7_data/'

# Изучение  валидационных данных 

Что бы понимать какие признаки нам нужны для обучения модели нам необходимо узнать какие признаки описывают наблюдения, для которых мы должны предсказать целевую переменную.  

Поэтому для начала исследуем набор данных эти наблюдения описывающие.

In [7]:
test = pd.read_csv(f'{path}test.csv')

### Общий обзор

In [13]:
test.sample(3,random_state=RANDOM_SEED)

,bodyType,brand,car_url,color,complectation_dict,description,engineDisplacement,enginePower,equipment_dict,fuelType,...,vehicleConfiguration,vehicleTransmission,vendor,Владельцы,Владение,ПТС,Привод,Руль,Состояние,Таможня
19026,внедорожник 5 дв.,TOYOTA,https://auto.ru/cars/used/sale/toyota/rav_4/11...,чёрный,NaN,ЛОТ: 01215604\nФаворит Хофф\n\nВы можете получ...,2.0 LTR,150 N12,"{""tinted-glass"":true,""airbag-driver"":true,""iso...",бензин,...,ALLROAD_5_DOORS AUTOMATIC 2.0,автоматическая,JAPANESE,3 или более,NaN,Оригинал,полный,Левый,Не требует ремонта,Растаможен
3337,седан,AUDI,https://auto.ru/cars/used/sale/audi/a6/1101268...,синий,NaN,Авто в идеальном состоянии проблем нет смотрит...,1.8 LTR,125 N12,"{""engine-proof"":true,""esp"":true,""airbag-driver...",бензин,...,SEDAN MECHANICAL 1.8,механическая,EUROPEAN,2 владельца,1 год и 5 месяцев,Оригинал,передний,Левый,Не требует ремонта,Растаможен
11807,внедорожник 5 дв.,NISSAN,https://auto.ru/cars/used/sale/nissan/x_trail/...,белый,NaN,Комфорт: 6. Безопасность: 4. Обзор: 2. Салон: 2.,2.0 LTR,141 N12,"{""electro-window-back"":true,""airbag-driver"":tr...",бензин,...,ALLROAD_5_DOORS VARIATOR 2.0,вариатор,JAPANESE,2 владельца,NaN,Оригинал,полный,Левый,Не требует ремонта,Растаможен


In [10]:
test.info(verbose=False)
describe_df(test)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34686 entries, 0 to 34685
Columns: 32 entries, bodyType to Таможня
dtypes: int64(6), object(26)
memory usage: 8.5+ MB


,count,unique,top,freq,min,max,type,NaN_prop
bodyType,34686,24,седан,13147,NaN,NaN,<class 'str'>,0.000
brand,34686,12,BMW,4473,NaN,NaN,<class 'str'>,0.000
car_url,34686,34686,https://auto.ru/cars/used/sale,1,NaN,NaN,<class 'str'>,0.000
color,34686,16,чёрный,11010,NaN,NaN,<class 'str'>,0.000
complectation_dict,6418,2364,"{""id"":""4562904"",""name"":""Elegan",51,NaN,NaN,<class 'float'>,0.815
description,34686,31732,Выгода до 82 000 руб. при обме,264,NaN,NaN,<class 'str'>,0.000
engineDisplacement,34686,55,2.0 LTR,7391,NaN,NaN,<class 'str'>,0.000
enginePower,34686,315,249 N12,1708,NaN,NaN,<class 'str'>,0.000
equipment_dict,24690,23705,"{""leather"":true}",108,NaN,NaN,<class 'str'>,0.288
fuelType,34686,5,бензин,28601,NaN,NaN,<class 'str'>,0.000


Всего в исходном валидационном датасете 32 колонки с разными типами данных (6 int, 26 object) и разыми типами признаков:   
-бинарные (в колонках `vendor`, `ПТС`, `Руль`);   
-категориальные (например в колонках `bodyType`, `brand`, `color`, `Владельцы`);  
-количественные (например в `enginePower` или `mileage`).  



Некоторын колонки содержат словари или длинные строки, содержащие сразу несколько признаков. 

Колонка `brand` содержит 12 уникальных значений. Это позволяет нам существенно сократить обьем работы по парсингу, ведь релевантными будут только данные по этим 12 брендам, поэтому только по ним и будем собирать информацию.

Колонка `car_url` содержит ссылки на обьявления о продаже авто на сайте ***auto.ru***, что как бы намекает, что валидационный набор спарсен с сайта ***auto.ru***. Воспользуемся этим же источником.

Три колонки  `priceCurrency`, `Состояние` и `Таможня` содержат только одно уникальное хначение, они ненесут никакой информации, поэтому исключим их из рассмотрения.  
  
Теперь следует ознакомиться с  каждым признак/колонку более детально. Это позволит нам понять в каком виде/формате содержится информация в колонках и соответсвенно сформировать набор данных для обучения в том же виде/формате. Так же детальное ознакомлене позволит окончательно определится с необходимость включения информации из колонок в рассмотрение моделью. В случае включения данных в рассмотрение моделью, проведем EDA. 